In [ ]:
import timeatlas as ta
from timeatlas import TimeSeries, Metadata
from timeatlas import models
from pandas import Series, DatetimeIndex
import pandas as pd
import os
from pathlib import Path
import fbprophet

---
# Models

How to create models with TimeAtlas? At first let's create a TimeSeries.

In [ ]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts

## Grammar

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
m = models.Prophet()

In [ ]:
m.fit(ts)

In [ ]:
Y_hat = m.predict('4 days')

In [ ]:
m.X_train.plot()
Y_hat.plot()